In [1]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from dateutil.parser import parse
import time
import os
import pickle

In [2]:
# Anzahl der angezeigten Zeilen in JNP einstellen
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
# Pickl-Dateipfade vorbereiten:
data_path_weather = '/home/paul/python_projects/masterthesis/wma/data/wetter/'

pkl_file_wetter_step_1 = 'daten_wetter_step_1.p'

In [4]:
df_wetter_pkl = pickle.load(open(data_path_weather + pkl_file_wetter_step_1, 'rb'))

In [5]:
df_wetter_pkl.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,DATE_TIME
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0036,1","01350,1,C,N","011265,1,N,1","+0130,1","+0100,1","99999,9",2007-09-30 07:20:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"210,1,N,0051,1","01200,1,C,N","011265,1,N,1","+0140,1","+0100,1","99999,9",2007-09-30 07:50:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"220,1,N,0051,1","01350,1,C,N","011265,1,N,1","+0140,1","+0110,1","99999,9",2007-09-30 08:20:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"230,1,N,0051,1","99999,9,C,N","011265,1,N,1","+0140,1","+0100,1","99999,9",2007-09-30 08:50:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"220,1,N,0067,1","01200,1,C,N","011265,1,N,1","+0150,1","+0100,1","99999,9",2007-09-30 09:20:00


In [6]:
df_wetter_1 = df_wetter_pkl.drop(columns=['SOURCE','LATITUDE','LONGITUDE','REPORT_TYPE','QUALITY_CONTROL','CIG','VIS']).copy()
df_wetter_1.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","200,1,N,0036,1","+0130,1","+0100,1","99999,9",2007-09-30 07:20:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","210,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 07:50:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0051,1","+0140,1","+0110,1","99999,9",2007-09-30 08:20:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","230,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 08:50:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0067,1","+0150,1","+0100,1","99999,9",2007-09-30 09:20:00


In [7]:
# WND Spalte beinhaltet mehrere Informationen die in einzelne Spalten getrennt werden müssen
df_wetter_1['WND_DA'] = None # WIND-OBSERVATION direction angle
df_wetter_1['WND_DA_QC'] = None # WIND-OBSERVATION direction quality code
df_wetter_1['WND_TC'] = None # WIND-OBSERVATION type code
df_wetter_1['WND_SR'] = None # WIND-OBSERVATION speed rate
df_wetter_1['WND_SR_QC'] = None # WIND-OBSERVATION speed quality code

df_wetter_1.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","200,1,N,0036,1","+0130,1","+0100,1","99999,9",2007-09-30 07:20:00,None,None,None,None,None
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","210,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 07:50:00,None,None,None,None,None
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0051,1","+0140,1","+0110,1","99999,9",2007-09-30 08:20:00,None,None,None,None,None
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","230,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 08:50:00,None,None,None,None,None
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0067,1","+0150,1","+0100,1","99999,9",2007-09-30 09:20:00,None,None,None,None,None


In [8]:
# Neue WND-Spalten befüllen
for i, row in df_wetter_1.iterrows():
    wnd_splitted = row['WND'].split(",")
    df_wetter_1.loc[i,['WND_DA']] = wnd_splitted[0]
    df_wetter_1.loc[i,['WND_DA_QC']] = wnd_splitted[1]
    df_wetter_1.loc[i,['WND_TC']] = wnd_splitted[2]
    df_wetter_1.loc[i,['WND_SR']] = wnd_splitted[3]
    df_wetter_1.loc[i,['WND_SR_QC']] = wnd_splitted[4]

df_wetter_1.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","200,1,N,0036,1","+0130,1","+0100,1","99999,9",2007-09-30 07:20:00,200,1,N,0036,1
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","210,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 07:50:00,210,1,N,0051,1
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0051,1","+0140,1","+0110,1","99999,9",2007-09-30 08:20:00,220,1,N,0051,1
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","230,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 08:50:00,230,1,N,0051,1
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0067,1","+0150,1","+0100,1","99999,9",2007-09-30 09:20:00,220,1,N,0067,1


In [9]:
# WND_DA_QC Datenqualität prüfen
'''
0 = Passed gross limits check
1 = Passed all quality control checks
2 = Suspect
3 = Erroneous
4 = Passed gross limits check, data originate from an NCEI data source
5 = Passed all quality control checks, data originate from an NCEI data source
6 = Suspect, data originate from an NCEI data source
7 = Erroneous, data originate from an NCEI data source
9 = Passed gross limits check if element is present
'''
df_wetter_1.groupby(['WND_DA_QC']).agg({
    'WND_DA_QC' : 'count'
})

,WND_DA_QC
WND_DA_QC,
1,474
5,80
9,64


In [10]:
# WND_TC Datenqualität prüfen
'''
A = Abridged Beaufort
B = Beaufort
C = Calm
H = 5-Minute Average Speed
N = Normal
R = 60-Minute Average Speed
Q = Squall
T = 180 Minute Average Speed
V = Variable
9 = Missing
(If a value of 9 appears with a wind speed of 0000, this indicates calm winds.)
'''
df_wetter_1.groupby(['WND_TC'],as_index=True).agg({
    'WND_TC' : 'count'    
})

,WND_TC
WND_TC,
9,2
C,10
N,444
V,162


In [11]:
df_wetter_1.loc[(df_wetter_1.WND_TC == '9')]

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC
367,2014,NewYork,2014-11-02,2014-11-02 13:30:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,9,9999,9","+0072,5","-0033,5","10113,5",2014-11-02 15:51:00,999,9,9,9999,9
379,2016,NewYork,2016-11-06,2016-11-06 13:30:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,9,9999,9","+0133,5","+0011,5","10228,5",2016-11-06 15:51:00,999,9,9,9999,9


In [12]:
# WND_SR_QC Datenqualität prüfen
'''
0 = Passed gross limits check
1 = Passed all quality control checks
2 = Suspect
3 = Erroneous
4 = Passed gross limits check, data originate from an NCEI data source
5 = Passed all quality control checks, data originate from an NCEI data source
6 = Suspect, data originate from an NCEI data source
7 = Erroneous, data originate from an NCEI data source
9 = Passed gross limits check if element is present
'''
df_wetter_1.groupby(['WND_SR_QC'],as_index=True).agg({
    'WND_SR_QC' : 'count'    
})

,WND_SR_QC
WND_SR_QC,
1,516
5,97
9,5


In [13]:
# WND_DA Datenqualität prüfen
'''
999 = Missing. If type code (below) = V, then 999 indicates variable wind direction
'''
df_wetter_1.loc[(df_wetter_1.WND_DA == '999')].groupby(['WND_DA','WND_DA_QC','WND_TC']).agg({
    'WND':'count'
})

WND
WND_DA WND_DA_QC WND_TC     
999    9         9         2
                 C        10
                 N         4
                 V        48

In [14]:
# WND_SR Datenqualität prüfen
'''
A general domain comprised of the numeric characters (0-9). 9999 = Missing.
'''
df_wetter_1.loc[(df_wetter_1.WND_SR == '9999')].groupby(['WND_SR']).agg({
    'WND':'count'
})

,WND
WND_SR,
9999,5


In [15]:
df_wetter_1.loc[(df_wetter_1.WND_SR == '9999')]

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC
367,2014,NewYork,2014-11-02,2014-11-02 13:30:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,9,9999,9","+0072,5","-0033,5","10113,5",2014-11-02 15:51:00,999,9,9,9999,9
379,2016,NewYork,2016-11-06,2016-11-06 13:30:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,9,9999,9","+0133,5","+0011,5","10228,5",2016-11-06 15:51:00,999,9,9,9999,9
394,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0122,5","-0011,5","10326,5",2018-11-04 17:51:00,999,9,N,9999,9
395,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0122,5","-0011,5","10320,5",2018-11-04 18:51:00,999,9,N,9999,9
396,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0128,5","-0017,5","10318,5",2018-11-04 19:51:00,999,9,N,9999,9


In [16]:
df_wetter_1.loc[(df_wetter_1.Ort == 'NewYork') & (df_wetter_1.Jahr == '2018')]

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC
394,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0122,5","-0011,5","10326,5",2018-11-04 17:51:00,999,9,N,9999,9
395,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0122,5","-0011,5","10320,5",2018-11-04 18:51:00,999,9,N,9999,9
396,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0128,5","-0017,5","10318,5",2018-11-04 19:51:00,999,9,N,9999,9
397,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72502014734,2.1,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US","999,9,V,0015,5","+0128,5","+0000,5","10324,5",2018-11-04 17:51:00,999,9,V,0015,5
398,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72502014734,2.1,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US","999,9,N,0015,1","+0128,1","+0000,1","10324,1",2018-11-04 18:00:00,999,9,N,0015,1
399,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72502014734,2.1,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US","210,5,N,0026,5","+0128,5","-0006,5","10319,5",2018-11-04 18:51:00,210,5,N,0026,5
400,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72502014734,2.1,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US","070,5,N,0021,5","+0133,5","-0006,5","10318,5",2018-11-04 19:51:00,070,5,N,0021,5


In [17]:
# TMP-Datenqualität prüfen
'''
A general domain comprised of the numeric characters (0-9), a plus sign (+), and a minus sign (-). +9999 = Missing.
'''
df_wetter_1.loc[(df_wetter_1.TMP == '9999,9')] # -> keine fehlendenn Werte


,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC


In [18]:
df_wetter_1.loc[(df_wetter_1.TMP == '+9999,9')] # -> -> keine fehlendenn Werte

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC


In [19]:
# DEW-Datenqualität prüfen
'''
A general domain comprised of the numeric characters (0-9), a plus sign (+), and a minus sign (-). +9999 = Missing.
'''
df_wetter_1.loc[(df_wetter_1.DEW == '9999,9')] # -> keine fehlendenn Werte

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC


In [20]:
df_wetter_1.loc[(df_wetter_1.DEW == '+9999,9')] # -> -> keine fehlendenn Werte

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC


In [21]:
# SLP-Datenqualität prüfen
'''
A general domain comprised of the numeric characters (0-9), a plus sign (+), and a minus sign (-). +9999 = Missing.
'''
df_wetter_1.loc[(df_wetter_1.SLP == '99999,9')].groupby(['SLP']).agg({'SLP':'count'}) 

,SLP
SLP,
"99999,9",443


<p>Auf grund der Analyse müssen unterschiedliche Dataframes angelegt werden</p>
<lu>
<li>Ein Dataframe mit den Temperaturen und Taupunkten, da hier alle Datenvorhanden sind</li>
<li>Dataframes für die Windstärke und Windrichtung, da hier einige Daten fehlen</li>
<li>Dataframe für den Luftdruck, da hier auch einige Daten fehlen</li>
</lu>

In [22]:
#df_wetter_1.loc[(df_wetter_1.SLP == '99999,9')]